## 1. 개발 환경 설정

- 1:1 문의 데이터의 경우 온라인에서 많이 사용하는 문장이 포함됨
- KcELECTRA DATASET


> **NOTE**
공개된 한국어 Transformer 계열 모델들은 대부분 한국어 위키, 뉴스 기사, 책 등 잘 정제된 데이터를 기반으로 학습한 모델입니다. 한편, 실제로 NSMC와 같은 User-Generated Noisy text domain 데이터셋은 정제되지 않았고 구어체 특징에 신조어가 많으며, 오탈자 등 공식적인 글쓰기에서 나타나지 않는 표현들이 빈번하게 등장합니다.
KcELECTRA는 위와 같은 특성의 데이터셋에 적용하기 위해, 온라인 뉴스에서 댓글과 대댓글을 수집해, 토크나이저와 ELECTRA모델을 처음부터 학습한 Pretrained ELECTRA 모델입니다.
기존 KcBERT 대비 데이터셋 증가 및 vocab 확장을 통해 상당한 수준으로 성능이 향상되었습니다.

- 따라서 kobert에 비해 정확도가 약 82% -> 83%로 증가하는 것을 확인할 수 있었음
- 또한 clean() 함수를 통한 preprocessing을 통해 NLP Model이 내용을 더 잘 이해할 수 있도록 처리함

### 1-1. 라이브러리 설치

In [ ]:
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget

### 1-2. 라이브러리 import

In [ ]:
from mecab import MeCab
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wget,os
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
import wget,os

### 1-3. 한글 글꼴 설정(Windows)

In [ ]:
# os.environ['JAVA_HOME'] = "C:\Program Files\Java\jdk-19"

### 1-3. 한글 글꼴 설정(Colab)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=FONT_PATH, size=10).get_name()
print(font_name)
plt.rcParams['font.family']=font_name
assert plt.rcParams['font.family'] == [font_name], "한글 폰트가 설정되지 않았습니다."

NanumGothic


### 1-4. 구글드라이브 연결(Colab)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 5. Using pre-trained model(Optional)
* 한국어 pre-trained model로 fine tuning 및 성능 분석
> * [BERT-tutorial](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)
> * [HuggingFace-Korean](https://huggingface.co/models?language=korean)

## 여기서부터 실행 KcELECTRA 구현
### MultiClassification

* 참고 blog : https://velog.io/@seolini43/KOBERT로-다중-분류-모델-만들기-파이썬Colab

In [7]:
from transformers import AutoTokenizer, AutoModel

# # 문장을 tokenize할 tokenizer와 model을 다운로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
# model = AutoModel.from_pretrained("beomi/KcELECTRA-base-v2022")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
!pip install gluonnlp

In [9]:
!pip3 install mxnet-mkl==1.6.0 numpy==1.23.1

In [10]:
!pip install soynlp emoji

In [11]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [12]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [13]:
#GPU 사용
device = torch.device("cuda:0")

In [14]:
import pandas as pd

df_test=pd.read_csv("/content/drive/MyDrive/aivle_school_mini_project_4/2023.04.03_미니프로젝트4차_실습자료/train.csv")
df_test.head()

,text,label
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드2
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,웹
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,코드2
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",코드2
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,코드2


In [15]:
max([len(x) for x in df_test['text'].values])

6012

In [16]:
label_dict = {
    '코드1': 0,
    '코드2': 1,
    '웹': 2,
    '이론': 3,
    '시스템 운영': 4,
    '원격': 5
}

df_test['label'] = df_test['label'].map(label_dict)

In [17]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = emoji.replace_emoji(x, replace='') #emoji 삭제
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x


In [18]:
x_train = [clean(x) for x in df_test['text'].values]

df_test['remove_special_sent'] = x_train

In [19]:
df_test.head()

,text,label,remove_special_sent
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",1,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ..."
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,2,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,1,glob.glob(PATH) 를 사용할 때 질문입니다. PATH에 [ ] 가 포함되...
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",1,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals..."
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,1,filename = TEST_IMAGE + str(round(frame_sec)) ...


In [20]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split


data = df_test[['remove_special_sent', 'label']]
dataset_train, dataset_valid = train_test_split(data, test_size=0.2, random_state=0)

# 내용 확인
print(dataset_train.iloc[0])
print(dataset_train.iloc[-1])

remove_special_sent    new로 새로운 객체를 만들고 그 새로 만들어진 객체에 값을 넣기위해 Person함...
label                                                                  2
Name: 2287, dtype: object
remove_special_sent    1. # D: embed_dim D = 100 이때 embed_dim이 무엇인지 방...
label                                                                  1
Name: 2732, dtype: object


In [21]:
print(len(dataset_train))
print(len(dataset_valid))

2964
742


In [22]:
# Setting parameters
max_len = 150
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-6

In [23]:
# train_data tokenize

encoded_train = tokenizer(
    dataset_train['remove_special_sent'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 확인
print( encoded_train[0].tokens )
print( encoded_train[0].ids )
print( encoded_train[0].attention_mask )
print()
print('디코딩 :',tokenizer.decode(encoded_train[0].ids))

['[CLS]', 'new', '##로', '새로운', '객체', '##를', '만들고', '그', '새로', '만들어진', '객체', '##에', '값을', '넣', '##기위해', 'Per', '##son', '##함', '##수에', '##서', 'this', '##를', '이용한', '##게', '맞나요', '?', '그럼', 'Per', '##son', '##이라는', '함수', '##는', '값을', '넣', '##기위한', '보조', '##역할', '##이라고', '생각해도', '될까요', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [24]:
# valid data tokenized

encoded_valid = tokenizer(
    dataset_valid['remove_special_sent'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

print('디코딩 :',tokenizer.decode(encoded_valid[0].ids))
print('디코딩 :',tokenizer.decode(encoded_valid[-1].ids))

디코딩 : [CLS] 실습 피드백 요청합니다. 손해 부분의 경우, 자사 서비스를 사용했을 때 발생하는 손해에 대해 작성하는 것인가요? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
디코딩 : [CLS] salesdata ['Age'] = pd. to _ datetime ( salesdata ['OrderYear'-'BirthYear'] ). dt. year 이렇게 주문일자 - 출생년도 구문 실행시켰으나 오류가 뜹니다. 어떤 방식으로 해야 AGE 연령 도출이 가능할까요? [SEP] [PAD] [PAD]

In [25]:
# args = {
#     'random_seed': 42, # Random Seed
#     'pretrained_model': 'beomi/KcELECTRA-base',  # Transformers PLM name
#     'pretrained_tokenizer': '',  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
#     'batch_size': 32,
#     'lr': 5e-6,  # Starting Learning Rate
#     'epochs': 1,  # Max Epochs
#     'max_length': 150,  # Max Length input size
#     'train_data_path': "nsmc/ratings_train.txt",  # Train Dataset file
#     'val_data_path': "nsmc/ratings_test.txt",  # Validation Dataset file
#     'test_mode': False,  # Test Mode enables `fast_dev_run`
#     'optimizer': 'AdamW',  # AdamW vs AdamP
#     'lr_scheduler': 'exp',  # ExponentialLR vs CosineAnnealingWarmRestarts
#     'fp16': True,  # Enable train on FP16(if GPU)
#     'tpu_cores': 0,  # Enable TPU with 1 core or 8 cores
#     'cpu_workers': os.cpu_count(),
# }

In [26]:
from torch.utils.data import Dataset, DataLoader

# 데이터셋 클래스
class ReviewDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = { key: torch.tensor(val[idx]) for key, val in self.encodings.items() }
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)


# 데이터셋 생성
train_dataset = ReviewDataset(encoded_train, dataset_train['label'].values)
valid_dataset = ReviewDataset(encoded_valid, dataset_valid['label'].values)

In [27]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

model = AutoModelForSequenceClassification.from_pretrained('beomi/KcELECTRA-base-v2022', num_labels=6)
model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [28]:
!pip install accelerate -U

In [29]:
training_args = TrainingArguments(
    output_dir='./temp/electra',
    overwrite_output_dir='True',

    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    logging_dir='./temp/logs',
    logging_steps=log_interval,
    evaluation_strategy="steps",
    eval_steps=log_interval,

    # https://discuss.huggingface.co/t/save-only-best-model-in-trainer/8442/8
    save_total_limit=2,
    save_strategy='no',
    load_best_model_at_end=False,
)

#정확도 측정을 위한 함수 정의
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # average: 'micro', 'macro', 'weighted' or 'samples'
    # 참고 https://aimb.tistory.com/152
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'acc': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
<ipython-input-26-9f6c135ef231>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { key: torch.tensor(val[idx]) for key, val in self.encodings.items() }


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall
200,0.886300,0.584843,0.811321,0.811308,0.813260,0.811321
400,0.222300,0.769648,0.808625,0.808662,0.811488,0.808625
600,0.079900,0.949312,0.820755,0.821350,0.830265,0.820755
800,0.032800,0.929327,0.824798,0.824478,0.827675,0.824798


<ipython-input-26-9f6c135ef231>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { key: torch.tensor(val[idx]) for key, val in self.encodings.items() }
<ipython-input-26-9f6c135ef231>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { key: torch.tensor(val[idx]) for key, val in self.encodings.items() }
<ipython-input-26-9f6c135ef231>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { key: torch.tensor(val[idx]) for key, val in self.encodings.items() }
<ipython-input-26-9f6c135ef231>:10: UserWarning: To copy constru

TrainOutput(global_step=930, training_loss=0.26482459986081686, metrics={'train_runtime': 760.1005, 'train_samples_per_second': 38.995, 'train_steps_per_second': 1.224, 'total_flos': 2284831571256000.0, 'train_loss': 0.26482459986081686, 'epoch': 10.0})

In [31]:
trainer.evaluate(valid_dataset)

<ipython-input-26-9f6c135ef231>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { key: torch.tensor(val[idx]) for key, val in self.encodings.items() }


{'eval_loss': 0.9351044297218323,
 'eval_acc': 0.8301886792452831,
 'eval_f1': 0.8290823676814335,
 'eval_precision': 0.8308362983472249,
 'eval_recall': 0.8301886792452831,
 'eval_runtime': 6.4919,
 'eval_samples_per_second': 114.297,
 'eval_steps_per_second': 3.697,
 'epoch': 10.0}

In [32]:
test_data=pd.read_csv("/content/drive/MyDrive/aivle_school_mini_project_4/2023.04.03_미니프로젝트4차_실습자료/test.csv")
test_data.head()

,id,text
0,0,팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이...
1,1,"셀프테스트에서 받은 점수가 해당 교육을 이수하고, 취업 연계등을 하는데 있어서 영향..."
2,2,a= int(input('정수 A의 값을 입력하시오.:'))\nb= int(inpu...
3,3,"def max4(a,b,c,d):\n maximum = a\n if b&..."
4,4,i**=2 가 i **=2 와 왜 같은지가 이해가 잘 안됩니다!!


In [34]:
test = [clean(x) for x in test_data['text'].values]

test_data['remove_special_sent'] = test

In [35]:
test_data.head()

,id,text,remove_special_sent
0,0,팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이...,팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이 ...
1,1,"셀프테스트에서 받은 점수가 해당 교육을 이수하고, 취업 연계등을 하는데 있어서 영향...","셀프테스트에서 받은 점수가 해당 교육을 이수하고, 취업 연계등을 하는데 있어서 영향..."
2,2,a= int(input('정수 A의 값을 입력하시오.:'))\nb= int(inpu...,a= int(input('정수 A의 값을 입력하시오.:')) b= int(input...
3,3,"def max4(a,b,c,d):\n maximum = a\n if b&...","def max4(a,b,c,d): maximum = a if b&gt;maximum..."
4,4,i**=2 가 i **=2 와 왜 같은지가 이해가 잘 안됩니다!!,i**=2 가 i **=2 와 왜 같은지가 이해가 잘 안됩니다!!


In [36]:
#토큰화
encoded_test = tokenizer(
    test_data['remove_special_sent'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

print('디코딩 :',tokenizer.decode(encoded_test[0].ids))
print('디코딩 :',tokenizer.decode(encoded_test[-1].ids))

디코딩 : [CLS] 팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이 경우에는 다운받아야 되는 파일이 있을까요? 노트북 트래픽이 많아지면 소리부분 끊기거나 전체화면을 왔다갔다 하니까 놓치는 부분이 있어서 질문드립니다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
디코딩 : [CLS] 안녕하세요 현재 aidu에서 테스트를 진행중입니다, 버전문제로 파이썬 3. 6에서 3. 7로 업그레이드를 하고싶은데 " ERROR : Could not find a version that satisfies the requirement python = = 3. 7 ( from versions : none ) ERROR : No matching distribution found for python = = 3. 7 " 해당 에러로 업그레이드를 

In [38]:
# 데이터셋 생성
# tokenizer, label(0)

zero_labels = np.zeros(len(test_data), dtype=int)
test_dataset = ReviewDataset(encoded_test, zero_labels)

In [39]:
# https://huggingface.co/course/en/chapter3/3?fw=pt#evaluation

# 예측하기
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

<ipython-input-26-9f6c135ef231>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { key: torch.tensor(val[idx]) for key, val in self.encodings.items() }


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(929, 6) (929,)


In [43]:
reverse_dict = {
    0:'코드1',
    1:'코드2',
    2:'웹',
    3:'이론',
    4:'시스템 운영',
    5:'원격'
}

In [ ]:
test_data['pred_target'] = np.array([ np.argmax(p) for p in predictions.predictions ], dtype='int8')

for text, label in zip(test_data['text'], test_data['pred_target']):

  print(f'input : {text}')
  print('----'*50)
  print(f'>> 해당 문의는 {reverse_dict[label]} 입니다.')
  print()